## Introduction

In this notebook, we will load all the data available to us, clean it and keep relevant information.

## Downloading the dataset

In [1]:
"""%%bash
wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470
' -O KuaiRec.zip
unzip KuaiRec.zip -d data_final_project"""

"%%bash\nwget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470\n' -O KuaiRec.zip\nunzip KuaiRec.zip -d data_final_project"

## Imports

In [2]:
import os
import pandas as pd
import ast

## Loading all the data

In [3]:
root = "data_final_project/KuaiRec 2.0/data/"
big_matrix = pd.read_csv(root + "big_matrix.csv").drop_duplicates().dropna()
small_matrix = pd.read_csv(root + "small_matrix.csv").drop_duplicates().dropna()
social_network = pd.read_csv(root + "social_network.csv").drop_duplicates().dropna()
user_features = pd.read_csv(root + "user_features.csv").drop_duplicates().dropna()
item_daily_features = pd.read_csv(root + "item_daily_features.csv").drop_duplicates().dropna()
item_categories = pd.read_csv(root + "item_categories.csv").drop_duplicates().dropna()
caption_category = pd.read_csv(root + "kuairec_caption_category.csv", lineterminator='\n').drop_duplicates().dropna()
# There is a category id equal to -124 and no category being 30, so I set the negative one to be 30 for practical reasons:
caption_category['first_level_category_id'] = [x if x >= 0 else 30 for x in caption_category['first_level_category_id']]

## Data Cleaning

##### Cleaning steps:
- small/big_matrix: watch_ratio (we remove unrealistic ones)
- social_network: change friend list to actual list type (not str), add a column for friends count
- item_categories: we explode feats and encode them
- item_daily_features: we keep only NORMAL videos because we do not want to recommend ads, we keep only videos which are in public visible status because we can't recommend a private video, we also apply cumulative sums on the data to get the stats up to a certain day
- user_features: good as-is
- caption_category: good as-is

## Creating filters for users and videos
Since we will be using a lot of data from different files, we have to make sure we don't have more missing values then necessary.

To prevent that, we create sets containing all the user and video ids for which we have data in all the sources we want.

Also, one obvious thing we have to consider is that we do not want our recomender system to recommend videos which are ads or which are not publicly visible, so we filter these one out from the start, right before we create our sets.

In [4]:
item_daily_features_cleaned = item_daily_features.copy()
item_daily_features_cleaned = item_daily_features_cleaned[item_daily_features_cleaned["visible_status"] == "public"]
item_daily_features_cleaned = item_daily_features_cleaned[item_daily_features_cleaned["video_type"] == "NORMAL"]

In [5]:
known_user_ids = set(user_features["user_id"].unique())
known_video_ids = set(caption_category["video_id"].unique()) & set(item_categories["video_id"].unique()) & set(item_daily_features_cleaned["video_id"].unique())
print(f"We have data for {len(known_user_ids)} users and {len(known_video_ids)} videos")

We have data for 6899 users and 8754 videos


We can see that we have data foor 6899 singular users, and 8754 singular videos.

## Big & Small matrix

These two files contain the same exact type of data, so we will clean them together, following these steps:
- We drop the "date" and "timestamp" columns since there is already the column "time". We also drop the "play_duration" column we will not need since we already have the watch ratio and the video duration.
- We drop any duplicate rows, then filter out unrealistically high watch ratios, as well as 0 watch ratio values since these do not describe an interaction.
- We assume that if a user interacted two times with the same video, then we can add the watch ratios because the user went back to watch it, then we filter again the outliers.
- We filter out videos for which we have incomplete or no data.
- We make sure times are in the right format.

In [6]:
def clean_big_small_matrix(df: pd.DataFrame) -> pd.DataFrame:
    cleaned = df.copy()

    cleaned = cleaned.drop(columns=["play_duration", "date", "timestamp"])
    # Remove duplicates, null values and filter out unrealistic watch_ratio values
    cleaned = cleaned.dropna().drop_duplicates()
    cleaned = cleaned[cleaned["watch_ratio"] < 5]
    cleaned = cleaned[cleaned["watch_ratio"] > 0]
    cleaned["watch_ratio"] = cleaned.groupby(["user_id", "video_id"])["watch_ratio"].transform("sum")
    cleaned = cleaned[cleaned["watch_ratio"] < 5]
    # Remove interactions with unknown users or unknown videos
    cleaned = cleaned[cleaned["user_id"].isin(known_user_ids)]
    cleaned = cleaned[cleaned["video_id"].isin(known_video_ids)]
    # Convert times to actual times
    cleaned["time"] = pd.to_datetime(cleaned["time"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
    assert(not cleaned.isnull().sum().any())
    return cleaned

small_matrix_cleaned = clean_big_small_matrix(small_matrix)
big_matrix_cleaned = clean_big_small_matrix(big_matrix)
big_matrix_cleaned

,user_id,video_id,video_duration,time,watch_ratio
0,0,3649,10867,2020-07-05 00:08:23.438,1.273397
2,0,5262,7908,2020-07-05 00:16:06.687,0.107613
3,0,1963,9590,2020-07-05 00:20:26.792,1.434307
4,0,8234,11000,2020-07-05 00:43:05.128,1.296455
5,0,8228,8576,2020-07-05 01:00:25.500,3.113806
...,...,...,...,...,...
12530799,7175,6597,8741,2020-09-05 06:35:01.104,1.004462
12530800,7175,6630,13855,2020-09-05 15:00:33.379,0.313389
12530803,7175,10360,7067,2020-09-05 19:10:29.041,1.253997
12530804,7175,10360,7067,2020-09-05 19:10:36.995,1.253997


## Social Network

By running a quick check we noticed that 93.5% of the users do not have any friends, so we believe it is safe to not take this data into account:

In [7]:
percent_who_have_friends = len(social_network["user_id"].unique()) / len(known_user_ids) * 100
print(f"{100 - percent_who_have_friends:.2f}% of users do not have friends")

93.16% of users do not have friends


## Item Categories

- We filter out videos for which we have incomplete or no data.
- We parse the "feat" column into a list of integers instead of a string.

In [8]:
item_categories_cleaned = item_categories.copy()
item_categories_cleaned = item_categories_cleaned[item_categories_cleaned["video_id"].isin(known_video_ids)]
item_categories_cleaned["feat"] = item_categories_cleaned["feat"].apply(ast.literal_eval)
item_categories_cleaned

,video_id,feat
0,0,[8]
2,2,[9]
4,4,[5]
5,5,[6]
6,6,[19]
...,...,...
10722,10722,[5]
10723,10723,[11]
10724,10724,[2]
10726,10726,[19]


## Item Daily Features

- We filter out videos for which we have incomplete or no data.
- (Private videos and advertisements have already been filtered out whem building or known_video_ids list so we do not have to do it)
- We ensure all the time variables have the right format.
- We define a "stats_cols" and "useful_cols" list to filter the columns we want from all the ones available. We assume that keeping them all may not be relevant, and we are also constrained by RAM memory issues.

In [9]:
item_daily_features_cleaned = item_daily_features_cleaned[item_daily_features_cleaned["video_id"].isin(known_video_ids)]
item_daily_features_cleaned["date"] = pd.to_datetime(item_daily_features_cleaned["date"].astype(str), errors="coerce")
item_daily_features_cleaned["upload_dt"] = pd.to_datetime(item_daily_features_cleaned["upload_dt"].astype(str), errors="coerce")
stats_cols = ["valid_play_cnt", "like_cnt", "comment_cnt", "share_cnt", "follow_cnt", "collect_cnt", "download_cnt"]
useful_cols = ["video_id", "date", "video_duration", "upload_dt"] + stats_cols
item_daily_features_cleaned = item_daily_features_cleaned[useful_cols]
item_daily_features_cleaned

,video_id,date,video_duration,upload_dt,valid_play_cnt,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,download_cnt
23,0,2020-07-28,5966.0,2020-03-30,4280,462,10,0,247,2.0,3
24,0,2020-07-29,5966.0,2020-03-30,3892,443,7,0,242,7.0,5
25,0,2020-07-30,5966.0,2020-03-30,3884,411,9,4,226,6.0,3
26,0,2020-07-31,5966.0,2020-03-30,4259,456,5,0,223,3.0,9
27,0,2020-08-01,5966.0,2020-03-30,8860,800,9,1,275,7.0,9
...,...,...,...,...,...,...,...,...,...,...,...
343335,10722,2020-09-05,9700.0,2020-09-05,143,4,1,0,1,0.0,0
343336,10723,2020-09-05,4833.0,2020-09-05,114,24,0,0,0,0.0,0
343337,10724,2020-09-05,54720.0,2020-09-05,754,264,29,1,8,0.0,1
343339,10726,2020-09-05,5132.0,2020-09-05,5382,44,0,1,2,0.0,2


## User Features

We believe that knowing if a user is streamer or a video author will not tell us much about what he likes, because anyone can produce content. Furthermore, features like the number of followers or the number of days since the user's first registration zill most likely not tend to change a user's behavior.
For the onehot_feat features, we do not know what they represent exactly and we do not have enough RAM memory to load them all alongside the other information, which is why we decide to not use them.
In conclusion, we will not use the user_features table for our project.

## Caption Category

- We filter out videos for which we have incomplete or no data.
- We keep only the "firt_level_category_id" column, since it defines the category of the video, which will most likely help us for our project.

In [10]:
caption_category_cleaned = caption_category.copy()
useful_cols = ["video_id", "first_level_category_id"]
caption_category_cleaned = caption_category_cleaned[useful_cols]
caption_category_cleaned

,video_id,first_level_category_id
0,0,8
2,2,9
3,3,26
4,4,5
5,5,6
...,...,...
10722,10722,5
10723,10723,33
10724,10724,6
10726,10726,38


## Saving the cleaned dataframes

In [11]:
export_dir = "./exports/cleaned_data/"
if not os.path.exists(export_dir):
    os.makedirs(export_dir)
big_matrix_cleaned.to_parquet(export_dir + "big_matrix_cleaned.pq")
small_matrix_cleaned.to_parquet(export_dir + "small_matrix_cleaned.pq")
item_categories_cleaned.to_parquet(export_dir + "item_categories_cleaned.pq")
item_daily_features_cleaned.to_parquet(export_dir + "item_daily_features_cleaned.pq")
caption_category_cleaned.to_parquet(export_dir + "caption_category_cleaned.pq")